* will add distribution of zmag_close and zmag_open (distribution over all beads, all cycles) -> look at datatable to show quantiles
* finish implementing fixed beads
* rewrite ramp.py library

In [33]:
import data
%run ramp_module
import bokeh.plotting
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import *
import time
output_notebook()


ramp called as main


Loading BokehJS ...

In [34]:
# Please use this cell to give me necessary information 
track_file="/media/data/helicon/remi/2016-11-08/ramp_5HPs_mix.trk" # test on time scale change not adding anything to this data
#track_file="/media/data/helicon/zhen/2016-12-02/ramp_mirB_3233_120_F1_M2A.trk"

In [36]:
track=data.Track(path=track_file)
mod=RampModel()
rpdata=RampData(model = mod)
rpdata.setTrack(track)

tmp = time.time()
keep = rpdata.areGoodBeadCycle()
print(time.time()-tmp)

rpdata.setTrack(track)
tmp = time.time()
rpdata.clean()
print(time.time()-tmp)



rpfulldata = RampData(model = mod)
rpfulldata.setTrack(track)

0.036699533462524414
0.1325395107269287


In [32]:
todel = set([i[0] for i in keep.keys() if isinstance(i[0],int) if not keep[i]])
print(todel)
goods=set(i for i in rpfulldata.beads() if i not in todel)
print(goods&rpdata.beads())
print("goods beads =",rpdata.beads())
print("new good beads =",goods)
print(keep[rpfulldata.bcids])

{0, 1, 4, 5, 9, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 30, 31, 32, 34, 36, 40, 44, 46, 48, 53, 54, 55}
{2, 3, 6, 7, 8, 10, 13, 14, 23, 24, 25, 26, 27, 28, 29, 33, 35, 37, 38, 39, 41, 42, 43, 45, 47, 49, 50, 51, 52}
goods beads = {2, 3, 6, 7, 8, 10, 13, 14, 23, 24, 25, 26, 27, 28, 29, 33, 35, 37, 38, 39, 41, 42, 43, 45, 47, 49, 50, 51, 52}
new good beads = {2, 3, 6, 7, 8, 10, 13, 14, 23, 24, 25, 26, 27, 28, 29, 33, 35, 37, 38, 39, 41, 42, 43, 45, 47, 49, 50, 51, 52}
(7, 3)       True
(31, 6)     False
(55, 5)      True
(42, 10)     True
(16, 9)      True
(19, 4)      True
(43, 3)      True
(46, 12)    False
(20, 7)     False
(44, 4)      True
(47, 9)      True
(21, 6)     False
(45, 9)      True
(8, 5)       True
(32, 2)     False
(9, 0)      False
(33, 3)      True
(36, 8)     False
(10, 7)      True
(34, 8)     False
(14, 1)      True
(38, 2)      True
(15, 4)     False
(39, 11)     True
(26, 12)     True
(50, 1)      True
(3, 2)       True
(27, 1)      True
(4, 5)      False
(28, 10

In [4]:
times = [i for i in rpdata.dataz.keys() if i[0]=="t"] 
zmags = [i for i in rpdata.dataz.keys() if i[0]=="zmag"]
print("zmags=",zmags)
fig = bokeh.plotting.figure(width=600, height=600)
fig.circle(rpdata.dataz[times[1]],rpdata.dataz[zmags[1]])
print("est=",(rpdata.dataz[zmags[1]]==-0.168).sum())
unit = rpdata._estimateUnitScale()
print("unit=",unit)
show(fig)


zmags= [('zmag', 10), ('zmag', 9), ('zmag', 17), ('zmag', 14), ('zmag', 7), ('zmag', 13), ('zmag', 6), ('zmag', 16), ('zmag', 4), ('zmag', 19), ('zmag', 11), ('zmag', 12), ('zmag', 2), ('zmag', 8), ('zmag', 5), ('zmag', 3), ('zmag', 15), ('zmag', 0), ('zmag', 1), ('zmag', 18)]
est= 0
unit= 41.5


In [5]:
uscaletest = rpdata.dzdt[list(rpdata.bcids)].apply(lambda x: _isdzdt_unit_consistent(x,5, 35))
print("done")
stdtest = rpdata.dzdt[list(rpdata.bcids)].apply(lambda x: _isGooddzdt(x,5))
print("finish")

function _isdzdt_unit_consistent is dubious
Utility of this function has not been proven
done
finish


In [6]:
uoverstd=[k for k,v in uscaletest[stdtest].items() if not v]
stdoveru=[k for k,v in stdtest[uscaletest].items() if not v]
print(len(uoverstd))
print(uoverstd[:3])
print(len(stdoveru))
print(stdoveru[:3])
print(len(rpdata.bcids))

0
[]
1513
[(31, 6), (89, 11), (92, 16)]
2180


In [36]:
badbcids = [i for i in rpdata.bcids if not stdtest[i]]
badbcids.sort()
for bcid in badbcids:
    print(bcid)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(2, 1)
(2, 18)
(2, 19)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(3, 13)
(3, 14)
(3, 15)
(3, 16)
(3, 17)
(3, 18)
(3, 19)
(4, 0)
(4, 1)
(4, 18)
(5, 0)
(5, 1)
(5, 6)
(5, 10)
(5, 14)
(5, 16)
(5, 17)
(5, 18)
(5, 19)
(6, 0)
(6, 1)
(6, 12)
(6, 14)
(7, 0)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 7)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(7, 16)
(7, 17)
(7, 18)
(7, 19)
(8, 1)
(8, 5)
(8, 8)
(8, 10)
(9, 0)
(9, 1)
(9, 2)
(9, 3)
(9, 4)
(9, 5)
(9, 7)
(9, 8)
(9, 9)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(9, 16)
(9, 17)
(9, 18)
(9, 19)
(10, 1)
(10, 14)
(11, 0)
(11, 1)
(11, 2)
(11, 3)
(11, 4)
(11, 5)
(11, 6)
(11, 8)
(11, 9)
(11

In [38]:
bcid=(89,11)
bid,cid=bcid
fig = bokeh.plotting.figure(width=500,height=500)
fig.line(x=rpdata.dataz["t",cid],y=rpdata.dzdt[bid,cid])
fig.circle(x=rpdata.dataz["t",cid][rpdata.det[bcid]],y=rpdata.dzdt[bid,cid][rpdata.det[bcid]],fill_color="red")
show(fig)
